# **Presentación del caso**

Predecir o estimar el precio de una vivienda puede ser de gran ayuda a la hora  de tomar decisiones importantes tales como la adquisición de casa propia . A continuación se presenta un dataset compuesto por **25, 660 registros** para **Argentina y Colombia** adjunto a las siguientes **10 variables**: info original [aquí](https://www.datasource.ai/es/home/data-science-competitions-for-startups/prediccion-de-precios-de-apartamentos-en-argentina-y-colombia)

    1.   pais : "Argentina", "Colombia"
    2.   provincia_departamento: Provincia o departamento (no ambas)  donde se ubica el departamento
    3.   ciudad: Ciudad donde se ubica el departamento
    4.   property_type: "Departamento", "Apartamento"
    5.   operation_type: "Venta", "Alquiler"
    6.   rooms: cantidad de espacios en general dentro del apartamento
    7.   bedrooms: cantidad de cuartos donde dormir dentro del apartamento
    8.   bathrooms: cantidad de baños dentro del apartamento
    9.   surface_total: área total en metros cuadrados del departamento
    10.  currency: USD (dólar americano)

![Image of Yaktocat](https://www.datasource.ai/uploads/7c2c64c37b855715637538ef4f19a46d.png)

# **Lectura de los datos**

In [ ]:
import pandas as pd
tipo_datos={'rooms': 'int', 'bedrooms': 'int', 'bathrooms': 'int', 'surface_total': 'int', 'price': 'int'}
train=pd.read_csv("https://raw.githubusercontent.com/javalpe/datasets/master/train_argcol.csv",sep=";", index_col='Id', dtype=tipo_datos)
test=pd.read_csv("https://raw.githubusercontent.com/javalpe/datasets/master/test_argcol.csv",sep=";", index_col='Id', dtype=tipo_datos)

In [ ]:
train.shape, test.shape

((25660, 11), (6416, 10))

# **Preprocesamiento de los datos**

## I. *Verificación de datos perdidos*

In [ ]:
for col in train.columns:
  print('En la columna',col,'hay',train[col].isna().sum(),'valores nulos')

En la columna pais hay 0 valores nulos
En la columna provincia_departamento hay 0 valores nulos
En la columna ciudad hay 0 valores nulos
En la columna property_type hay 0 valores nulos
En la columna operation_type hay 0 valores nulos
En la columna rooms hay 0 valores nulos
En la columna bedrooms hay 0 valores nulos
En la columna bathrooms hay 0 valores nulos
En la columna surface_total hay 0 valores nulos
En la columna currency hay 0 valores nulos
En la columna price hay 0 valores nulos


In [ ]:
for col in test.columns:
  print('En la columna',col,'hay',test[col].isna().sum(),'valores nulos')

En la columna pais hay 0 valores nulos
En la columna provincia_departamento hay 0 valores nulos
En la columna ciudad hay 0 valores nulos
En la columna property_type hay 0 valores nulos
En la columna operation_type hay 0 valores nulos
En la columna rooms hay 0 valores nulos
En la columna bedrooms hay 0 valores nulos
En la columna bathrooms hay 0 valores nulos
En la columna surface_total hay 0 valores nulos
En la columna currency hay 0 valores nulos


## II. *Verificación de outliers*

Para el presente caso, la variable más susceptible a error de valores muy por encima del promedio es la **columna surface_total**

In [ ]:
outliers_train = train[train['surface_total'] > train['surface_total'].quantile(.95)].index
train.loc[outliers_train,'surface_total'] = train['surface_total'].quantile(.95)

In [ ]:
outliers_test = test[test['surface_total'] > test['surface_total'].quantile(.95)].index
test.loc[outliers_test,'surface_total'] = test['surface_total'].quantile(.95)

## III. *Merge con data externa*

In [ ]:
pais_provincia=pd.read_csv("https://raw.githubusercontent.com/javalpe/datasets/master/pais_provincia_argcol.csv",sep=";")

Añadimos **nuevas columnas** al final con ayuda del método **merge**. Previamente **concatenamos el país y provincia**

In [ ]:
train['pais_provincia'] = train['pais']+ "_" + train['provincia_departamento']
test['pais_provincia'] = test['pais']+ "_" + test['provincia_departamento']

In [ ]:
train= pd.merge(train, pais_provincia, on='pais_provincia', how='left')
test= pd.merge(test, pais_provincia, on='pais_provincia', how='left')

In [ ]:
provincia_ciudad=pd.read_csv("https://raw.githubusercontent.com/javalpe/datasets/master/provincia_ciudad_argcol.csv",sep=";")

Añadimos **nuevas columnas** al final con ayuda del método **merge**. Previamente **concatenamos la provincia y ciudad**

In [ ]:
train['provincia_ciudad'] = train['provincia_departamento']+ "_" + train['ciudad']
test['provincia_ciudad'] = test['provincia_departamento']+ "_" + test['ciudad']

In [ ]:
train= pd.merge(train, provincia_ciudad, on='provincia_ciudad', how='left')
test= pd.merge(test, provincia_ciudad, on='provincia_ciudad', how='left')

## VI. *Preparamos los datos para el modelo*

Al tratarse de un modelo de **regresión** se requiere exclusivamente variables **numéricas**.

In [ ]:
train_regresion = train.loc[:,['rooms', 'bedrooms', 'bathrooms', 'surface_total','price',
                                'promedio_provincia', 'mediana_provincia','percentil10_provincia', 'percentil25_provincia','percentil75_provincia', 'percentil90_provincia',
                                'promedio_ciudad', 'mediana_ciudad', 'percentil10_ciudad','percentil25_ciudad', 'percentil75_ciudad', 'percentil90_ciudad']]
test_regresion = test.loc[:,['rooms', 'bedrooms', 'bathrooms', 'surface_total',
                                'promedio_provincia', 'mediana_provincia','percentil10_provincia', 'percentil25_provincia','percentil75_provincia', 'percentil90_provincia',
                                'promedio_ciudad', 'mediana_ciudad', 'percentil10_ciudad','percentil25_ciudad', 'percentil75_ciudad', 'percentil90_ciudad']]

Vamos a verificar la **multicolinearidad** utilizando el método del **Variance Inflation Factor(VIF)**. Para mayor información consultar la siguiente [página](https://towardsdatascience.com/multi-collinearity-in-regression-fe7a2c1467ea)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif["features"] = train_regresion.columns
vif["vif_Factor"] = [variance_inflation_factor(train_regresion.values, i) for i in range(train_regresion.shape[1])]
vif

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,features,vif_Factor
0,rooms,34.843577
1,bedrooms,34.628884
2,bathrooms,15.575305
3,surface_total,19.347542
4,price,3.292542
5,promedio_provincia,1685.049054
6,mediana_provincia,2965.048558
7,percentil10_provincia,1405.652831
8,percentil25_provincia,3393.510908
9,percentil75_provincia,3603.473506


Se evidencia que los datos externos superan por mucho el límite permisible. Así como también algunas variables originales. Para corregirlo haremos uso del **PCA** (sin considerar el precio).

In [ ]:
X = train_regresion.drop('price',axis=1)
y = train_regresion['price']

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components=6)
components=pca.fit_transform(X)
X=pd.DataFrame(data=components,columns=['PCA1','PCA2','PCA3','PCA4','PCA5','PCA6'])

vif = pd.DataFrame()
vif["features"] = X.columns
vif["vif_value"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif

,features,vif_value
0,PCA1,1.0
1,PCA2,1.0
2,PCA3,1.0
3,PCA4,1.0
4,PCA5,1.0
5,PCA6,1.0


In [ ]:
pca = PCA(n_components=6)
components=pca.fit_transform(test_regresion)
X_test=pd.DataFrame(data=components,columns=['PCA1','PCA2','PCA3','PCA4','PCA5','PCA6'])

Finalmente, con ayuda de la librería **train_test_split** dividimos la data de **train**: 85% para entrenamiento y **15%** para validación

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid= train_test_split(X,y,test_size = 0.15,random_state=1) 

# **Modelamiento**

Importamos las librerías de los modelos de Regresión:
1.   Lasso
2.   Ridge
3.   ElasticNet


In [ ]:
from sklearn.metrics import mean_squared_log_error
import numpy as np

In [ ]:
from sklearn.linear_model import Lasso
for i in range(90,99):
    lasso = Lasso(alpha=i/10000)
    lasso.fit(X_train,y_train)
    lasso_predictions=lasso.predict(X_valid)
    print("Mi RMSLE es: ", np.sqrt(mean_squared_log_error(y_valid,abs(lasso_predictions))), "cuando alpha es: ", i/10000)

Mi RMSLE es:  0.7341915823917559 cuando alpha es:  0.009
Mi RMSLE es:  0.7341915823917368 cuando alpha es:  0.0091
Mi RMSLE es:  0.7341915823917178 cuando alpha es:  0.0092
Mi RMSLE es:  0.7341915823916987 cuando alpha es:  0.0093
Mi RMSLE es:  0.7341915823916797 cuando alpha es:  0.0094
Mi RMSLE es:  0.7341915823916606 cuando alpha es:  0.0095
Mi RMSLE es:  0.7341915823916415 cuando alpha es:  0.0096
Mi RMSLE es:  0.7341915823916224 cuando alpha es:  0.0097
Mi RMSLE es:  0.7341915823916033 cuando alpha es:  0.0098


In [ ]:
from sklearn.linear_model import Ridge
for i in range(1,10):
    ridge = Ridge(alpha=i/100)
    ridge.fit(X_train,y_train)
    ridge_predictions=ridge.predict(X_valid)
    print("Mi RMSLE es: ", np.sqrt(mean_squared_log_error(y_valid,abs(ridge_predictions))), "cuando alpha es: ", i/100)

Mi RMSLE es:  0.7341915823934715 cuando alpha es:  0.01
Mi RMSLE es:  0.7341915823934714 cuando alpha es:  0.02
Mi RMSLE es:  0.7341915823934714 cuando alpha es:  0.03
Mi RMSLE es:  0.7341915823934713 cuando alpha es:  0.04
Mi RMSLE es:  0.7341915823934713 cuando alpha es:  0.05
Mi RMSLE es:  0.7341915823934712 cuando alpha es:  0.06
Mi RMSLE es:  0.7341915823934712 cuando alpha es:  0.07
Mi RMSLE es:  0.7341915823934709 cuando alpha es:  0.08
Mi RMSLE es:  0.7341915823934709 cuando alpha es:  0.09


In [ ]:
from sklearn.linear_model import ElasticNet
for i in range(5,15):
    elastic = ElasticNet(alpha=i/10000,l1_ratio=0.45)
    elastic.fit(X_train,y_train)
    elastic_predictions=elastic.predict(X_valid)
    print("Mi RMSLE es: ", np.sqrt(mean_squared_log_error(y_valid,abs(elastic_predictions))), "cuando alpha es: ", i/10000)

Mi RMSLE es:  0.7341915823933863 cuando alpha es:  0.0005
Mi RMSLE es:  0.7341915823933695 cuando alpha es:  0.0006
Mi RMSLE es:  0.7341915823933524 cuando alpha es:  0.0007
Mi RMSLE es:  0.7341915823933354 cuando alpha es:  0.0008
Mi RMSLE es:  0.7341915823933183 cuando alpha es:  0.0009
Mi RMSLE es:  0.7341915823933012 cuando alpha es:  0.001
Mi RMSLE es:  0.7341915823932842 cuando alpha es:  0.0011
Mi RMSLE es:  0.7341915823932672 cuando alpha es:  0.0012
Mi RMSLE es:  0.7341915823932498 cuando alpha es:  0.0013
Mi RMSLE es:  0.7341915823932329 cuando alpha es:  0.0014


# **Cross Validation**

In [ ]:
from sklearn.model_selection import cross_val_score
ridge = Ridge(alpha=0.08)
scores = cross_val_score(ridge, X, y, cv=5)

In [ ]:
scores

array([0.10226985, 0.11480796, 0.08793059, 0.09388797, 0.08594255])

# **Generando nuestras soluciones**

In [ ]:
ridge = Ridge(alpha=0.08)
ridge.fit(X_train,y_train)
predicciones=abs(ridge.predict(X_test))

In [ ]:
origin = pd.read_csv("https://raw.githubusercontent.com/javalpe/datasets/master/test_argcol.csv", sep=";")
origin_id=origin['Id']

In [ ]:
respuestas_dict = {'id':origin_id,'price':predicciones}
respuestas = pd.DataFrame(respuestas_dict)

In [ ]:
respuestas.to_csv("james_valencia.csv",index=False)

![Image of Yaktocat](https://i.imgur.com/BaAad3q.png)